In [19]:
import pandas as pd

In [20]:
df=pd.read_csv(r'C:\Users\Mujahid\Desktop\MLDS\oneneuron\machine_learning_project\insurance-premium-prediction\notebooks\data\insurance.csv')

In [21]:
df.head()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   expenses  1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [23]:
df['region'].unique()

array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)

In [24]:
df.duplicated().sum()

1

In [25]:
df.drop_duplicates()

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.9,0,yes,southwest,16884.92
1,18,male,33.8,1,no,southeast,1725.55
2,28,male,33.0,3,no,southeast,4449.46
3,33,male,22.7,0,no,northwest,21984.47
4,32,male,28.9,0,no,northwest,3866.86
...,...,...,...,...,...,...,...
1333,50,male,31.0,3,no,northwest,10600.55
1334,18,female,31.9,0,no,northeast,2205.98
1335,18,female,36.9,0,no,southeast,1629.83
1336,21,female,25.8,0,no,southwest,2007.95


In [26]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
expenses    0
dtype: int64

In [27]:
X=df.drop(labels=['expenses'] , axis=1)

In [28]:
Y=df['expenses']

In [29]:
numerical_columns = df.columns[df.dtypes != 'object']
categorical_columns = df.columns[df.dtypes == 'object']


In [30]:
categorical_columns


Index(['sex', 'smoker', 'region'], dtype='object')

In [31]:
numerical_columns=numerical_columns[:-1]


In [32]:
numerical_columns


Index(['age', 'bmi', 'children'], dtype='object')

In [33]:
sex_categories=['male' , 'female']
smoker_categories=['yes' , 'no']
region_categories=['southwest', 'southeast', 'northwest', 'northeast']

In [34]:

from sklearn.preprocessing import StandardScaler  # HAndling Feature Scaling
from sklearn.preprocessing import OneHotEncoder  # OneHot Encoding
# pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [35]:
# Numerical Pipeline
num_pipeline = Pipeline(
    steps=[
        ('scaler', StandardScaler())

    ]

)

# Categorigal Pipeline
cat_pipeline = Pipeline(
    steps=[
        ('OneHotencoder', OneHotEncoder(categories=[
         sex_categories, smoker_categories, region_categories])),
        
        
    ]

)

preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, numerical_columns),
    ('cat_pipeline', cat_pipeline, categorical_columns)
])


In [36]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 Index(['age', 'bmi', 'children'], dtype='object')),
                                ('cat_pipeline',
                                 Pipeline(steps=[('OneHotencoder',
                                                  OneHotEncoder(categories=[['male',
                                                                             'female'],
                                                                            ['yes',
                                                                             'no'],
                                                                            ['southwest',
                                                                             'southeast',
                                                                             'northwest',
                                                                             'northeast']]))]),
                                 Index(['sex', 'smoker', 'region'], dtype='object'))])

In [37]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=42)

In [38]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [39]:
X_train.head()


,num_pipeline__age,num_pipeline__bmi,num_pipeline__children,cat_pipeline__sex_male,cat_pipeline__sex_female,cat_pipeline__smoker_yes,cat_pipeline__smoker_no,cat_pipeline__region_southwest,cat_pipeline__region_southeast,cat_pipeline__region_northwest,cat_pipeline__region_northeast
0,1.544465,0.109490,-0.915011,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.481874,-0.491191,-0.915011,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,1.048589,0.226289,1.560279,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,1.119429,0.026062,-0.915011,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,-1.005753,2.495529,0.735182,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [40]:
## Model Training

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
from sklearn.tree import DecisionTreeRegressor 
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [41]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [42]:
regression.coef_

array([  3688.37975328,   2091.4217928 ,    514.37695752,     52.49762358,
          -52.49762358,  11813.9472978 , -11813.9472978 ,   -329.52528225,
         -375.08035908,    109.06784463,    595.5377967 ])

In [43]:
regression=DecisionTreeRegressor()
regression.fit(X_train , y_train)
y_pred=regression.predict(X_test)
r2_score(y_test , y_pred)

0.7257003120178452

In [44]:
regression = SVR()
regression.fit(X_train, y_train)
y_pred = regression.predict(X_test)
r2_score(y_test, y_pred)


-0.08016069378908397

In [45]:
regression = AdaBoostRegressor()
regression.fit(X_train, y_train)
y_pred = regression.predict(X_test)
r2_score(y_test, y_pred)


0.7809147411673443

In [46]:
regression = SGDRegressor()
regression.fit(X_train, y_train)
y_pred = regression.predict(X_test)
r2_score(y_test, y_pred)


0.7698424767995036

In [47]:
import numpy as np


def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square


In [48]:
# Train multiple models

models = {
    'LinearRegression': LinearRegression(),
    'DecisionTree': DecisionTreeRegressor(),
    'AdaboostRegressor': AdaBoostRegressor(),
    'SGDRegressor': SGDRegressor(),
    'Lasso':Lasso(),
    'ElasticNet':ElasticNet(),
    'SVR':SVR(),
    'Ridge': Ridge(),
    
}
trained_model_list = []
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    # Make Predictions
    y_pred = model.predict(X_test)

    mae, rmse, r2_square = evaluate_model(y_test, y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:", rmse)
    print("MAE:", mae)
    print("R2 score", r2_square*100)

    r2_list.append(r2_square)

    print('='*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 5811.806354382953
MAE: 4144.886409993459
R2 score 76.96351080608883


DecisionTree
Model Training Performance
RMSE: 6484.453149953091
MAE: 2950.86435323383
R2 score 71.32253653046628


AdaboostRegressor
Model Training Performance
RMSE: 5210.161055675205
MAE: 4122.393365543961
R2 score 81.48616801042563


SGDRegressor
Model Training Performance
RMSE: 5816.030155555781
MAE: 4161.994493979257
R2 score 76.93001453946049


Lasso
Model Training Performance
RMSE: 5811.875809930442
MAE: 4145.20085522969
R2 score 76.96296019533814


ElasticNet
Model Training Performance
RMSE: 8427.199571103976
MAE: 6342.126392269282
R2 score 51.56485777120411


SVR
Model Training Performance
RMSE: 12584.824803989319
MAE: 8239.509225331525
R2 score -8.016069378908398


Ridge
Model Training Performance
RMSE: 5813.1731363103645
MAE: 4150.84395074168
R2 score 76.9526743956862


